# <font color='darkblue'>Modul 6: Übungsmaterial zu k-Means - Wie ergeben sich die besten Trainingsgruppen?</font> &nbsp; <font size='6'>&#9917;</font>

<hr style="border:1px solid gray"> </hr>

Im zweiten Video dieses Moduls haben Sie gesehen, dass der k-Means ein weitverbreitetes und effizientes Verfahren zur Clusterananlyse ist. Der k-Means soll nun auf einen dreidimensionalen Datensatz angewendet werden und im Anschluss ein weiteres Verfahren zur Bestimmung des k, d.h. der Anzahl der zu bestimmenden Cluster, thematisiert werden. 

## <font color='darkblue'>Inhalt</font>


1. [Anwendung von k-Means auf einen Fußball-Datensatz](#kap1)  
    1.1 [Datensatz einlesen und erforschen](#kap11)  
    1.2 [Anwendung von k-Means](#kap12)    
    

2. [Bestimmung von k mit dem Silhouettenkoeffizienten](#kap2)    
    2.1 [Bestimmung des Silhouettenkoeffizienten](#kap21)    
    2.2 [Bedeutung des Silhouettenkoeffizienten](#kap22)     
    2.3 [Anwendung auf das Fußballbeispiel](#kap23)


3. [Fazit](#kap3)

<hr style="border:1px solid gray"> </hr>

## <font color='darkblue'><p style='margin-bottom:-10px'>1. Anwendung von k-Means auf einen Fußball-Datensatz </p> <a name="kap1"></a>

In diesem Beispiel wird k-Means eingesetzt, um möglichst passend Trainingsgruppen beim Fußball zu bilden. Die Grundlage dafür bildet ein selbsterstellter Datensatz, der wichtige Fähigkeiten der einzelnen Spieler enthält. 

### <font color='darkblue'>1.1 Datensatz einlesen und erforschen</font> <a name="kap11"></a>

Zunächst werden die benötigten Bibliotheken eingebunden:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

Der folgende Datensatz enthält Daten zu Charakteristiken von 200 Fußballspielern. Der Datensatz wird nun zunächst eingelesen, erste Informationen ausgegeben und Histogramme geplottet: 

In [ ]:
df = pd.read_csv('M6_Uebung_Fussball.csv', sep=',')
df.info()

In [ ]:
df.hist(figsize=(12,9));

Der Datensatz enthält die folgenden Merkmale: 

<table align='left'>
    <thead>
        <tr>
          <th style="text-align:left">Spaltenname</th>
          <th style="text-align:left">Bedeutung</th>
          <th style="text-align:left">Weitere Informationen</th>
        </tr>
    </thead>
    <tr>
    <td style="text-align:left">100m Sprint</td>
    <td style="text-align:left">Zeit über 100m Sprintstrecke</td>
    <td style="text-align:left">[s]</td>
    </tr>
    <tr>
    <td style="text-align:left">10km Langlauf</td>
    <td style="text-align:left">Zeit über 10km Langlaufstrecke</td>
    <td style="text-align:left">[min]</td>
    </tr>
    <tr>
    <td style="text-align:left">Score Dribbling</td>
    <td style="text-align:left">Fähigkeiten beim Dribbling</td>
    <td style="text-align:left">-</td>
    </tr>
    <tr>
    <td style="text-align:left; vertical-align: top">Fussballer ID</td>
    <td style="text-align:left; vertical-align: top">ID Nummer</td>
    <td style="text-align:left; vertical-align: top">-</td>
    </tr>
</table>

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 

Um sich einen weiteren Überblick über die Daten zu schaffen, geben Sie die Korrelationsmatrix der Variablen aus. Versuchen Sie außerdem einen Grund dafür zu finden, dass der 100m Sprint und die FussballerID eine hohe Korrelation haben. 
    
Tipp: Geben Sie zur Beantwortung der Frage die letzten 10 Einträge des Datensatzes aus.
   
</div>

In [ ]:
# leeren

In [ ]:
# leeren

Die FussballerID ist offenbar nicht aussagekräftig für eine weitere Modellierung. Da die Merkmale außer der Ausnahme untereinander nicht korrelieren, werden nun zunächst alle drei Merkmale in einen Plot ausgegeben.  

In [ ]:
# Erstellen der Grafik
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot der Datenpunkte / Scatterplot
ax.scatter(df['100m Sprint'], df['10km Langlauf'], df['Score Dribbling'])

# Beschriftung der Achsen
ax.set_xlabel('100m Sprint')
ax.set_ylabel('10km Langlauf')
ax.set_zlabel('Score Dribbling')

plt.show()

Die Daten bilden - anders als in den bisherigen Beispielen - keine sofort ins Auge springenden Cluster. Hier wird k-Means trotzdem gute Dienste leisten. 

Da die Merkmale wieder sehr unterschiedliche Größenordnungen in ihren Ausprägungen aufweisen, wird vor der nachfolgenden Modellierung zunächst eine Skalierung vorgenommen: 

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(df)
df_scaled = pd.DataFrame(scaler.transform(df),columns=df.columns )

print(df_scaled)
df=df_scaled

### <font color='darkblue'>1.2 Anwendung von k-Means</font> <a name="kap12"></a>

Nun sollen Trainingsgruppen zusammengestellt werden, sodass Fußballer mit ähnlichen Eigenschaften in einer Gruppe sind und das Training auf ihre Stärken und Schwächen abgestimmt werden kann. Da nicht festgelegt ist, wie viele Trainingsgruppen es geben soll, ist die erste Aufgabe die Bestimmung der besten Anzahl.

Zunächst einmal wird die Ellenbogenmethode angewendet, um das beste k zur Anwendung des k-Means-Verfahren zu bestimmen.

In [ ]:
from sklearn.cluster import KMeans

y = []
for i in range(1, 15):
    kmeans = KMeans(n_clusters = i, random_state = 0)
    kmeans.fit(df)
    y.append(kmeans.inertia_)

plt.plot(range(1, 15), y)
plt.title('Ellenbogenplot', fontsize = 20)
plt.xlabel('Anzahl Cluster')
plt.ylabel('Inertia')
plt.xticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
plt.grid()
plt.show()

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 

Das Ergebnis ist nicht völlig eindeutig (kein klarer Knick!). Entscheiden Sie sich für einen Wert für k und wenden Sie für dieses k das k-Means-Verfahren an. Visualisieren Sie die zugehörige Clusterung.

Tipp: Zum Plotten definieren Sie eine Variable `y_pred` und nutzen den vorgegebenen Baustein.
</div>

In [ ]:
# Ergänzen Sie den Code um das passende k
k = #
kmeans = KMeans(n_clusters = k, random_state = 0)
kmeans.fit(df)

In [ ]:
# Definieren Sie eine Variable y_pred, die in dem nachfolgenden Code als color-Vektor übergeben wird

In [ ]:
# Erstellen der Grafik
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot der Datenpunkte / Scatterplot
ax.scatter(df['100m Sprint'], df['10km Langlauf'], df['Score Dribbling'], c = y_pred)

# Beschriftung der Achsen
ax.set_xlabel('100m Sprint')
ax.set_ylabel('10km Langlauf')
ax.set_zlabel('Score Dribbling')

plt.show()

<hr style="border:1px solid gray"> </hr>

## <font color='darkblue'>2. Bestimmung von k mit dem Silhouettenkoeffizienten</font> <a name="kap2"></a>

Eine Schwierigkeit des k-Means liegt in der Vorgabe des k, d.h. der Anzahl der zu bestimmenden Cluster. Eine Lösungsmethode, die bereits thematisiert wurde, ist die Ellenbogenmethode. Wie im vorhergehenden Beispiel zu sehen ist, ist diese Methode nicht immer eindeutig. 

Hier wird nun eine andere Methode gezeigt, deren Berechnung zwar aufwendiger ist, bei der die Bestimmung des k aber einfacher gelingt. Dies ist die Berechnung des Silhouettenkoeffizienten. 

Zunächst wird der Datensatz aus dem zweiten Video erzeugt: 

In [ ]:
from sklearn import cluster, datasets, mixture
blob_centers = np.array(
    [[1,  2],
     [4 , 5],
     [3,  1],
     [2.2,  2.8],
     ])
blob_std = np.array([0.3, 0.2, 0.4, 0.2])
X_sp, y_sp = datasets.make_blobs(n_samples=500, centers=blob_centers, cluster_std=blob_std, random_state=7)  

### <font color='darkblue'>2.1 Bestimmung des Silhouettenkoeffizienten</font> <a name="kap21"></a>

Der Silhouettenkoeffizient lässt sich mit einem Befehl aus der scikit-learn Bibliothek bestimmen. Hier wird zunächst die "optimale" Clusterung aus dem Videonotebook genutzt:

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score

k = 4
kmeans_sp = KMeans(n_clusters=k, random_state=0)

y_sp_pred = kmeans_sp.fit_predict(X_sp)
silhouette = silhouette_score(X_sp, y_sp_pred)
print("Für k =", k, "Cluster beträgt der Silhouettenkoeffizient:", silhouette)

Der Silhouettenkoeffizient liegt immer zwischen -1 und 1. Je näher er an 1 liegt, desto besser ist die Clusterung. 

### <font color='darkblue'>2.2 Bedeutung des Silhouettenkoeffizienten</font> <a name="kap22"></a>

Um zu verstehen, was der Silhouettenkoeffizient berechnet, wird zunächst der Silhouettenplot betrachtet. Er visualisiert die Aufteilung der Cluster. Dies ist besonders bei höherdimensionalen Daten hilfreich, bei denen eine optische Kontrolle über den Scatterplot nicht möglich ist. 

Der Silhouettenplot wird im folgenden Code umgesetzt:

<div class="alert alert-block alert-info">
&#128204; <b>Anmerkung:</b> Führen Sie die zwei Codezellen zunächst nur aus, um dann den zugehörigen, darunterliegenden Text zu lesen. Hier folgt dann ein Arbeitsauftrag der sich mit dem Verständnis des Codes befasst.

In [ ]:
#Code im Orginal aus: https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py
#Hier leicht verändert

import matplotlib.cm as cm

def plot_silhouette(X, y_pred, kmeans, k):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # Das linke Bild ist der Silhouettenplot.
    # Der Silhouettenkoeffizient kann Werte zwischen -1 und 1 annehmen.
    # Die Anzeige wird auf den gängigen Bereich zwischen [-0.1, 1] beschränkt.
    ax1.set_xlim([-0.1, 1])
    # Das (k+1)*10 fügt leeren Raum zwischen die Silhouettenplots von zwei Clustern ein, damit man sie gut unterscheiden kann. 
    ax1.set_ylim([0, len(X) + (k + 1) * 10])

    # Für jeden Punkt der Stichprobe wird die Silhouette berechnet.
    sample_silhouette_values = silhouette_samples(X, y_pred)

    y_lower = 10
    for i in range(k):
        # Die Silhouetten für alle Punkte, die zum Cluster i gehören, werden zusammengefasst und der Größe nach sortiert.
        ith_cluster_silhouette_values = sample_silhouette_values[y_pred == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / k)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Die Silhouettenplots werden in der Mitte mit den Clusternummern beschriftet.
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Ein neues y_lower für den Start der nächsten Silhouette wird berechnet. 
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("Silhouettenplot für die verschiedenen Cluster")
    ax1.set_xlabel("Werte der Silhouettenkoeffizienten")
    ax1.set_ylabel("Cluster Nummer")

    # Vertikale Linie für den Silhouettenkoeffizienten, d.h. den durchschnittlichen Silhouettenwert über alle Punkte
    ax1.axvline(x=silhouette_score(X_sp, y_sp_pred), color="red", linestyle="--")

    ax1.set_yticks([])  # keine Zahlen an der y-Achse
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # Das rechte Bild zeigt die aktuell berechneten Cluster. 
    colors = cm.nipy_spectral(y_pred.astype(float) / k)
    ax2.scatter(
        X[:, 0], X[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors, edgecolor="k"
    )

    # Beschriftung der Cluster
    centers = kmeans.cluster_centers_
    # Weiße Kreise um die Clusterzentren
    ax2.scatter(
        centers[:, 0],
        centers[:, 1],
        marker="o",
        c="white",
        alpha=1,
        s=200,
        edgecolor="k",
    )

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker="$%d$" % i, alpha=1, s=50, edgecolor="k")

    ax2.set_title("Visualisierung der geclusterten Daten")
    ax2.set_xlabel("Wertebereich des ersten Merkmals")
    ax2.set_ylabel("Wertebereich des zweiten Merkmals")

    plt.suptitle(
        "Silhouetten-Analyse für k-Means Clusteranalyse mit Anzahl der Cluster k = %d"
        % k,
        fontsize=14,
        fontweight="bold",
    )

    plt.show()

In [ ]:
plot_silhouette(X_sp, y_sp_pred, kmeans_sp, 4) 

Die Berechnung des Silhouttenkoeffizienten erfolgt in mehreren Schritten:

1. Für jeden Punkt wird seine Distanz zu seinem eigenen Cluster berechnet. Dies ist das arithmetische Mittel über die Abstände zu allen anderen Punkten seines Clusters.


2. Für jeden Punkt wird seine Distanz zu seinem nächstgelegenen Cluster berechnet. Dies ist das arithmetische Mittel der Abstände zu allen Punkten des nächstgelegenen Clusters.


3. Für jeden Punkt wird seine Silhouette berechnet. Dies ist die Differenz aus seiner Distanz zum nächstgelegenen Cluster und seiner Distanz zum eigenen Cluster, geteilt durch den größeren der beiden Werten. Hieraus folgt, dass die Silhouette eines Punktes immer Werte zwischen -1 und 1 annimmt:
    - Silhouette nahe 1 bedeutet, dass der Punkt in seinem Cluster liegt. 
    - Silhouette nahe 0 bedeutet, dass der Punkt fast so weit vom eigenen Cluster entfernt ist wie vom nächstgelegenen Cluster. 
    - Eine negative Silhouette bedeutet, dass der Punkt weiter vom eigenen Cluster entfernt ist als vom nächstgelegenen Cluster (Achtung: Nicht vom Zentrum, sondern von allen Punkten. Sonst wäre er im anderen Cluster.)
    
    
4. Der Silhouettenkoeffizient ist dann das arithmetische Mittel über die Silhouetten aller Punkte. 

Für den Silhouettenplot werden die Silhouetten der Punkte eines Clusters der Größe nach geordnet und als Balken nebeneinander geplottet. Auf diese Weise entsteht meistens wie oben links eine charakteristische Messerform. Der Silhouettenkoeffizient entspricht der roten gestrichelten Linie in der linken Darstellung.

Auf der rechten Seite ist zur Orientierung die zugehörige Clusterung dargestellt. 

Der Silhouettenkoeffizient wird also umso größer, je näher die Punkte eines Clusters beieinander liegen und desto weiter sie von den anderen Clustern entfernt liegen. Insgesamt bedeutet dies, wie oben bereits notiert, dass ein hoher Silhouettenkoeffizient eine gute Clusterung anzeigt.

<div class="alert alert-block alert-success"> <a name="p1"></a>
&#128187; <b>Arbeitsauftrag:</b> 

Lesen Sie den obigen Code und die Kommentierung durch und beantworten Sie die folgenden Fragen: 
    
1. Was passiert in der Schleife <span style='font-family:monospace'>for i in range(k): ... </span>?  
2. In welcher Zeile wird die Silhouette eines Punktes berechnet?
3. An welcher Stelle wird der Silhouettenkoeffizient bestimmt?
     
      
<font size="1">Die Antworten zu diesen Fragen finden Sie [am Ende](#answers) des Notebooks.</font>                                                                                                                     
</div>

Wie zu sehen, beträgt der Silhouettenkoeffizient im Beispiel bei 4 Clustern etwa 0,74. Wenn eine andere Anzahl an Clustern vorgegeben wird, wird der Wert sinken. Probieren Sie es aus!

Auch die Form der "Messer" ist aufschlussreich:
- Ein flaches/stumpfes Messer zeigt an, dass die Punkte des Clusters alle eng beieinander liegen. Das Cluster ist kompakt. Dies ist z.B. bei Cluster 1 der Fall. 
- Bei einem scharfen Messer liegen manche Punkte nahe beieinander, andere haben aber einen größeren Abstand zu den anderen Punkten. Das Cluster ist nicht kompakt und ein Teil der Punkte liegt recht nahe am nächsten Cluster. Cluster 2 und 3 haben diese Form.

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 

Im gelben Cluster sind wenige negative Silhouetten zu erkennen. Überlegen Sie, zu welchen Punkten der rechten Grafik sie gehören.  
    
</div>

### <font color='darkblue'>2.3 Anwendung auf das Fußballbeispiel</font> <a name="kap23"></a>

Nun soll der Silhouettenplot auch für das oben betrachtete Beispiel erstellt werden. 

Da es sich bei dem Beispiel um einen dreidimensionalen Datensatz handelt, wird aus dem Code von oben der Teil des zweidimensionalen Plots auf der rechten Seite herausgelöscht. 

In [ ]:
#Code orginal aus: https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py
#hier überarbeitet

def plot_silhouette_nd(X, y_pred, kmeans, k):
    fig = plt.figure()
    ax = fig.add_subplot(111)

    ax.set_xlim([-0.1, 1])
    ax.set_ylim([0, len(X) + (k + 1) * 10])

    sample_silhouette_values = silhouette_samples(X, y_pred)

    y_lower = 10
    for i in range(k):
        ith_cluster_silhouette_values = sample_silhouette_values[y_pred == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / k)
        ax.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        y_lower = y_upper + 10 

    ax.set_title("Silhouetten-Analyse für k-Means Clusteranalyse mit Anzahl der Cluster k = %d" % k,)
    ax.set_xlabel("Die Werte des Silhouettenkoeffizienten")
    ax.set_ylabel("Cluster Nummer")

    ax.axvline(x=silhouette_score(X, y_pred), color="red", linestyle="--")

    ax.set_yticks([])  
    ax.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    plt.show()

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 

Erstellen Sie den Silhouettenplot, indem Sie die Eingabevariablen der Funktion `plot_silhouette` in der nachfolgenden Zelle ergänzen. 
  
</div>

<div class="alert alert-block alert-info">
&#128204; <b>Anmerkung:</b>

Da unklar ist, ob Sie `y_pred` oben korrekt belegt haben, wird dieses hier neu definiert. Stimmen die Definitionen überein, haben Sie die obige Aufgabe richtig gelöst.

</div>

In [ ]:
y_pred = kmeans.fit_predict(df)

In [ ]:
# Ergänzen Sie die Eingabevariablen
plot_silhouette_nd(#, #, #)

Wie interpretieren Sie diesen Plot?
Ist die Aufteilung in drei Cluster geeignet für diese Datenmenge?

Eher nicht denn die "Messer" von Cluster 0 und 1 sind scharf. Der Silhouettenkoeffizient liegt nur bei etwa 0,4.

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b>   
    
Erstellen Sie analog zur Ellenbogenkurve eine Grafik, in der Sie auf der x-Achse k (variierend zwischen 2 und 8) auftragen und auf der y-Achse den Wert des zugehörigen Silhouettenkoeffizienten. 
    
Wählen Sie anschließend das beste k aus und visualisieren Sie die zugehörige Clusterung.

</div>

In [ ]:
# Ergänzen Sie den Code der Schleife
silhouette = []

for k in range(2,8):
    #
    #
    #
    
plt.plot(number_clusters,silhouette_avg, 'o' )
plt.xlabel('Anzahl der Cluster (k)', labelpad=10)
plt.ylabel('Silhouettenkoeffizient', labelpad=10)
plt.grid()

In [ ]:
# Ergänzen Sie das ausgewählte k

k = #
kmeans = KMeans(n_clusters = k, random_state = 0)
kmeans.fit(df)

y_pred = kmeans.fit_predict(df)

# Erstellen der Grafik
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot der Datenpunkte / Scatterplot
ax.scatter(df['100m Sprint'], df['10km Langlauf'], df['Score Dribbling'], c = y_pred)

# Beschriftung der Achsen
ax.set_xlabel('100m Sprint')
ax.set_ylabel('10km Langlauf')
ax.set_zlabel('Score Dribbling')

plt.show()

Der Silhouettenplot ergibt k=2 als günstigste Anzahl der Cluster. Hier ist aber nun erkennbar, dass auch k=7 eine gute Teilung ergeben würde. Dies war auf dem Ellenbogenplot nicht zu erkennen. 

<hr style="border:1px solid gray"> </hr>

## <font color='darkblue'>3. Fazit</font> <a name="kap3"></a>

Dieses Notebook hat:
- die Anwendung des k-Means-Algorithmus auf höherdimensionale Daten gezeigt.
- Silhouettenplot und seine Interpretation thematisiert.
- die Bestimmung des besten k mit dem Silhouettenkoeffizienten gezeigt.


<hr style="border:1px solid gray"> </hr>
<hr style="border:8px solid gray"> </hr>

<div class="alert alert-block alert-warning"> <a name="answers"></a>
    &#128161; <b>Antworten:</b>  

1. Was passiert in der Schleife <span style='font-family:monospace'>for i in range(k): ... </span>?  
    <br>*Es werden die einzelnen Cluster durchlaufen.*
2. In welcher Zeile wird die Silhouette eines Punktes berechnet?
    <br>*In der Zeile nach dem Schleifenbeginn:* <span style='font-family:monospace'>ith_cluster_silhouette_values = sample_silhouette_values[y_pred == i]</span>
3. An welcher Stelle wird der Silhouettenkoeffizient bestimmt?
    <br>*Er wird erst in der Zeile* <span style='font-family:monospace'>ax1.axvline(x=silhouette_score(X_sp, y_sp_pred), color="red", linestyle="--")</span> *berechnet.*  

<font size="1">&emsp;[Zurück](#p1).</font>
</div>